In [1]:
import sys  
sys.path.insert(0, '../OMPbenders/')
%matplotlib widget

In [2]:
import pandas as pd
import gurobipy   as      gp
from   gurobipy   import GRB
#from MasterProblemOMP import MasterProblem
#from SubProblemOMP import SubProblem
sys.path.insert(0, '../AuxiliarCodes/')
%matplotlib widget
from plotDrawpointsPoints import plotDrawpointsPoints
from plotGurobiOpenPitSolution import plotGurobiOpenPitSolution
from plotIntegratedOmpSolution import plotIntegratedOmpSolution
from plotOmpOpenPitSolution import plotOmpOpenPitSolution
from plotUndergroundSolution import plotUndergroundSolution

In [3]:
#path = "C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/undergroundModel/" #Notebook
path = "/home/williams/Tesis-Magister/Databases/"
#path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/undergroundModel/" #Desktop
undergroundDatabaseName = 'Modelo_F_OG.xlsx'
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = 'modelo_reblogeado.xlsx'

In [4]:
if undergroundDatabaseName == openPitDatabaseName:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = undergroundMineDataframe
else:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = pd.read_excel(path + openPitDatabaseName, engine="openpyxl") #Notebook

In [5]:
import sys  
sys.path.insert(0, '../AuxiliarCodes/')
import gurobipy   as     gp
from   gurobipy   import GRB
from drawpointFunction  import drawpointFunction
from globalFunctions import getNumberOfBlocksInADimension
from itertools import chain
from functools import reduce


class MasterProblem:
    #Underground Model + Crown Pillar Restrictions.
    def __init__(self, database, numberOfPeriods, colHeight, minColHeight,pos_x,pos_y,pos_z,pos_x_f,pos_y_f):
        self.database = database
        self.numberOfPeriods = numberOfPeriods
        self.DP_init = 0       #### Tipo de extracción
        self.desc = 0.1
        self.colHeight = colHeight#630#300
        self.minColHeight = minColHeight#0.20
        self.pos_x = pos_x     
        self.pos_y = pos_y   
        self.pos_z = pos_z
        self.pos_x_f = pos_x_f     
        self.pos_y_f = pos_y_f  
        self.p_t = 3791.912
        self.orientationToExtractTheDrawpoints = 0

    def setParameters(self):
        self.getUndergroundVariablesFromCSV()
        self.getUndergroundInfo()
        self.setUndergroundParameters()
        self.setUndergroundMineLimits()
        self.setUndergroundVariables()
        #self.setVandB_vParameters()
    
    def getUndergroundVariablesFromCSV(self):
        self.undergroundBlocksLenght = self.database['X'].to_dict()             
        self.undergroundBlocksWidth  = self.database['Y'].to_dict()             
        self.undergroundBlocksHeight = self.database['Z'].to_dict()             
        self.undergroundBlockTonnage = self.database['Ton'].to_dict()              
        self.undergroundBlockMineral  = self.database['Mineral'].to_dict()          
        self.undergroundBlockRecovery  = self.database['Recuperación'].to_dict()     
        self.undergroundCopperLaw  = self.database['%Cu'].to_dict()
        self.undergroundExtractionFixedCosts = self.database['CPlanta CA'].to_dict()
        self.undergroundVariableExtractionCosts = self.database['CMina CA'].to_dict()
        self.undergroundCP_S = self.database['CPlanta S'].to_dict()
        self.undergroundCM_S = self.database['CMINA S'].to_dict() 
    
    def getUndergroundInfo(self):
        self.undergroundBlocks = [i for i in range(len(self.undergroundBlocksLenght.values()))]

    def setUndergroundParameters(self):
        #Underground Parameters
        self.t_S   = {period : period + 1 for period in range(self.numberOfPeriods)}
        self.MU_mt = {period : 25806600.0/1  for period in range(self.numberOfPeriods)} #Tonleage es mina
        self.ML_mt = {period : 0/3  for period in range(self.numberOfPeriods)}
        self.MU_pt = {period : 17777880.0/1  for period in range(self.numberOfPeriods)}#Mineral es planta
        self.ML_pt = {period : 0/3 for period in range(self.numberOfPeriods)}
        self.qU_dt = {period : 1 for period in range(self.numberOfPeriods)}
        self.qL_dt = {period : 0.0001 for period in range(self.numberOfPeriods)}
        self.A_d   = {period : 2 for period in range(self.numberOfPeriods)}
        self.NU_nt = {period : 59 for period in range(self.numberOfPeriods)} 
        self.NL_nt = {period : 0 for period in range(self.numberOfPeriods)}
        self.N_t   = {period : 59* (1 + period) for period in range(self.numberOfPeriods)}
        self.RL_dt = {period : 0.2 for period in range(self.numberOfPeriods)}
        self.RU_dt = {period : 1 for period in range(self.numberOfPeriods)}

    def setUndergroundMineLimits(self):
        self.undergroundBlocksLenghtLimits = getNumberOfBlocksInADimension(self.undergroundBlocksLenght)
        self.undergroundBlocksWidthLimits = getNumberOfBlocksInADimension(self.undergroundBlocksWidth)
        self.undergroundBlocksHeightLimits = getNumberOfBlocksInADimension(self.undergroundBlocksHeight)

    def setUndergroundVariables(self):
        self.drawpoint, self.G_d, self.Q_d,self.q_d, self.C_pdt, self.C_mdt, self.predecessor, self.x_draw,self.y_draw, self.z_draw, self.drawpoints_blocks = drawpointFunction(
                        self.pos_x, self.pos_y, self.pos_z, self.colHeight, self.DP_init, self.undergroundBlocksLenghtLimits, self.undergroundBlocksWidthLimits, self.undergroundBlocksHeightLimits, self.undergroundBlockTonnage, self.undergroundCP_S, self.undergroundCM_S, self.undergroundBlockMineral,
                        self.undergroundCopperLaw, self.pos_x_f, self.pos_y_f,self.orientationToExtractTheDrawpoints)
        self.predecessorDict = {}
        self.predecessorDict[0] = []
        self.predecessorDict[1] = [0]
        for i in range(1,len(self.predecessor)):
            if self.predecessor[i][0] not in self.predecessorDict.keys():
                self.predecessorDict[self.predecessor[i][0]] = []
            self.predecessorDict[self.predecessor[i][0]].append(self.predecessor[i][1])
    """
    def setVandB_vParameters(self):
        self.setPossibleHeights()
        self.V = [height for height in chain(range(self.minHeight,self.maxHeight,self.blockHeight), [self.maxHeight])]
        self.B_v = {}
        self.rho_v = {v: (v - self.minHeight)/(self.maxHeight - self.minHeight) for v in self.V}

        for v in self.V:
            numberOfBlocksBelowV = (self.undergroundBlocksLenghtLimits[3]*self.undergroundBlocksWidthLimits[3])*((v-self.minHeight)/self.undergroundBlocksHeightLimits[0])
            blocksBelowV = [block for block in range(int(numberOfBlocksBelowV)) if not numberOfBlocksBelowV == 0]
            self.B_v[v] = blocksBelowV
    def setPossibleHeights(self):
        self.blockHeight, self.maxHeight, self.minHeight, self.numOfDifferentsBlocks = self.undergroundBlocksHeightLimits
    """
    """
    def addThetaRestriction(self, subProblemObjValue, estimatedW_v, pi_vb):
        self.undergroundModel.addConstr(self.theta <= subProblemObjValue + gp.quicksum(gp.quicksum((self.w_v[v]-estimatedW_v[v]) * pi_vb[b] for b in self.B_v) for v in self.V))
    """
    def setMathematicalModel(self):                
        self.undergroundModel = gp.Model(name = 'Modelo Integrado')
        self.undergroundModel.Params.TimeLimit = 3600
        self.undergroundModel.Params.OutputFlag = 0

        # Underground  Model

              #14. Naturaleza de las variables
        self.x_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="x_d")
        self.y_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.CONTINUOUS, name="y_d")
        self.z_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="z_d")

        #1. Restricción sobre la cantidad de tonelaje máxima y mínima a extraer en cada periodo.
        Ton_Up = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti]*self.G_d[d] for d in self.drawpoint) <= self.MU_mt[ti] for ti in self.t_S),
                                            "Min_max")
        
        Ton_low = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] * self.G_d[d] for d in self.drawpoint) >= self.ML_mt[ti] for ti in self.t_S),
                                            "Min_min")
        #2. Restricción sobre la cantidad de material máxima y mínima a procesar en cada periodo.
        Mat_Up = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] * self.Q_d[d] for d in self.drawpoint) <= self.MU_pt[ti] for ti in self.t_S),
                                            "Mat_max")

        Mat_low = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] * self.Q_d[d] for d in self.drawpoint) >= self.ML_pt[ti] for ti in self.t_S)
                                            , "Mat_min")

        
        #3. Rango de leyes máximas y mínimas a procesar
        GQC_low = self.undergroundModel.addConstrs((gp.quicksum(self.Q_d[d] * self.q_d[d] * self.y_dt[d, ti] for d in self.drawpoint) >=
                                self.qL_dt[ti] * gp.quicksum(self.G_d[d] * self.y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_low")
        
        GQC_Up = self.undergroundModel.addConstrs((gp.quicksum(self.Q_d[d] * self.q_d[d] * self.y_dt[d, ti] for d in self.drawpoint) <=
                                self.qU_dt[ti] * gp.quicksum(self.G_d[d] * self.y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_Up")

        #4. Todos los puntos de extracción deben ser iniciados en el largo de la extracción
        Drawp_init = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for ti in self.t_S) == 1 for d in self.drawpoint), "Drawp_init")

        #5. Los puntos de extracción deben ser activados al menos en el mismo periodo para que se inicie la extracción 
        Drawpextract_61 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, tau] for tau in range(ti+1)) >= self.z_dt[d, ti]  
                                            for d in self.drawpoint for ti in self.t_S), "Drawpextract_61")


        #6. Existe una cantidad máxima y mínima de drawpoints a abrir en cada periodo.
        Drawpextract_64_1 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for d in self.drawpoint) <= self.NU_nt[ti] for ti 
                                                        in self.t_S)
                                                        ,"Drawpextract_64_1")
        Drawpextract_64_2 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for d in self.drawpoint) >= self.NL_nt[ti] for ti 
                                                        in self.t_S)
                                                        , "Drawpextract_64_2")

        #7. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        Drawpextract_65 = self.undergroundModel.addConstrs((gp.quicksum(self.z_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")


        #8. Si iniciamos la extracción de un drawpoint esta debe durar por su duraci ́on determinada.
        ## Un drawpoint solamente puede ser extraido por un preiodo pre determinado (A_d)
        Drawpextract_62 = self.undergroundModel.addConstrs((gp.quicksum(self.z_dt[d, ti] for ti in self.t_S)  <= self.A_d[ti]  for d in self.drawpoint
                                                    for ti in self.t_S), "Drawp_62")

        ## Una vez se inicia extrayendo de un drawpoint, se continua extrayendo sin interrupción
        Drawpextract_63 = self.undergroundModel.addConstrs((self.A_d[ti] *(self.z_dt[d, ti] - self.z_dt[d, ti+1]) 
                                            - gp.quicksum(self.z_dt[d, tau] for tau in range(ti+1)) <= 0 
                                            for d in self.drawpoint for ti in range(0,max(self.t_S))), "Drawpextract_63")

        #9. Relación de variables, el porcentaje a extraer es 0 si no se extra un drawpoint.
        Drawpextract_66 = self.undergroundModel.addConstrs((self.y_dt[d, ti] <= self.z_dt[d, ti] for d in self.drawpoint for ti in self.t_S),
                                                    "Drawpextract_66")

        #10. Existe una tasa m ́ınima de extracci ́on para cada drawpoint a extraer.
        Drawpextract_67_1 = self.undergroundModel.addConstrs((self.RL_dt[ti] * self.z_dt[d, ti]  <=  self.y_dt[d, ti] for d in self.drawpoint
                                                        for ti in self.t_S), "Drawpextract_67_1")

        #11. La altura a extraer debe ser mayor a una cantidad m ́ınima.
        rest_11 = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d,ti] for ti in self.t_S)>= self.minColHeight for d in self.drawpoint))

        #12. No podemos extraer más del 100 % de un drawpoint.
        Reserver_cnst = self.undergroundModel.addConstrs((gp.quicksum(self.y_dt[d, ti] for ti in self.t_S) <= 1 for d in self.drawpoint),
                                                    "Reserver_cnst")

        #13. Si se activa un drawpoint, se extrae en ese periodo
        rest_13 = self.undergroundModel.addConstrs(self.x_dt[d,ti] <= self.z_dt[d, ti] for d in self.drawpoint for ti in self.t_S)

        #14. Naturaleza de variables.

        #15. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        rest_15 = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")
        
        #16. Restricción sobre el inicio de la extracci ́on de los drawpoints.


        alternative = self.undergroundModel.addConstrs(gp.quicksum(self.x_dt[a,s] for s in range(0,ti+1)) >= self.x_dt[d, ti] for d in self.drawpoint for ti in self.t_S for a in self.predecessorDict[d])
        #resta_prec = self.undergroundModel.addConstrs((gp.quicksum(self.x_dt[self.predecessor[l][0], m]*(max(self.t_S)-m+1) for m in self.t_S) <=
        #                            gp.quicksum(self.x_dt[self.predecessor[l][1], m]*(max(self.t_S)-m+1) for m in self.t_S)  
        #                            for l in range(len(self.predecessor))), "DP_Sup")

        
        #Conjuntos para el crown pillar

        #Restricciones del crown pillar
        #Variable 1 si y solo si el crown pillar esta ubicado en la elevaci ́on v, 0 en otro caso.

        
        self.w_v = self.undergroundModel.addVars(self.V,lb=self.fixed_w_v , ub=self.fixed_w_v  ,vtype=GRB.BINARY, name="w")

        
        self.u_t = self.undergroundModel.addVars(self.t_S,lb=self.fixed_u_t , ub=self.fixed_u_t  ,vtype=GRB.BINARY, name="u")


        ##Código experimental
        #natu = self.undergroundModel.addConstr(gp.quicksum(self.u_t[t] for t in self.t_S) <= 4)
        time_rest = self.undergroundModel.addConstrs(self.u_t[t-1]<= self.u_t[t] for t in range(1, len(self.t_S)))
        limit_time = self.undergroundModel.addConstrs(self.y_dt[d,t] <= self.u_t[t] for d in self.drawpoint for t in self.t_S)

        pillar_2 = self.undergroundModel.addConstrs(gp.quicksum(self.y_dt[d, ti] 
                                                        for ti in self.t_S) <= self.rho_v[v] * self.w_v[v] + (1 - self.w_v[v]) for v in self.V for d in self.drawpoint)
       
        pillar_3 = self.undergroundModel.addConstr(gp.quicksum(self.w_v[v] for v in self.V) == 1)


         #Función objetivo
        self.undergroundObjectiveFunction = self.subProblemObjValue + gp.quicksum(self.y_dt[d, ti]*((((self.p_t * self.q_d[d] - self.C_pdt[d] ) * self.Q_d[d])-(self.C_mdt[d]*self.G_d[d]))/
                                        ((1+self.desc)**(self.t_S[ti]))) for ti in self.t_S for d in self.drawpoint)

        self.undergroundModel.setObjective(self.undergroundObjectiveFunction, GRB.MAXIMIZE)
        self.undergroundModel.Params.MIPGap = 0.01

        self.undergroundModel.optimize()


    def optimizeModel(self, fixed_w_v, fixed_u_t, subProblemObjValue):
        self.fixed_w_v = fixed_w_v
        self.fixed_u_t = fixed_u_t
        self.subProblemObjValue = subProblemObjValue
        self.setMathematicalModel()
        
        if self.undergroundModel.Status == 2:
            print(self.undergroundModel.Status)
            self.lista_variable_Integrado = (self.undergroundModel.getAttr(GRB.Attr.X, self.undergroundModel.getVars()))
            solucion = self.undergroundModel.objVal
            runtime = self.undergroundModel.Runtime
            gap_f = self.undergroundModel.MIPGap

            self.x_dt_values = self.undergroundModel.getAttr('X', self.x_dt)
            self.y_dt_values = self.undergroundModel.getAttr('X', self.y_dt)
            self.z_dt_values = self.undergroundModel.getAttr('X', self.z_dt)
            return solucion
        else:
            print('The model cannot be solved because it is unbounded')
            return 0

In [6]:
import gurobipy   as     gp
from   gurobipy   import GRB
from itertools import chain
from globalFunctions import getNumberOfBlocksInADimension
from openPitFunctions import finalBlock


class SubProblem:
   #OpenPit Problem 
   def __init__(self, database, minHeightUnderground, maxHeightUnderground,numberOfPeriods, safetyLevel):
      self.database = database
      self.numberOfPeriods = numberOfPeriods
      self.minHeightUnderground = minHeightUnderground
      self.maxHeightUnderground = maxHeightUnderground
      self.safetyLevel = safetyLevel
      self.numberOfDestinations = 1
      self.basePrice = 3791.912
      self.desc = 0.1

   def setParameters(self):
      self.setOpenPitVariables()
      self.getOpenPitInfo()
      self.setOpenPitParameters()
      self.setOpenPitMineLimits()
      self.setPossibleHeights()
      self.setHeightSets()

   def setOpenPitVariables(self):
      self.openPitBlocksLength = self.database['X'].to_dict() 
      self.openPitBlocksWidth = self.database['Y'].to_dict() 
      self.openPitBlocksHeight = self.database['Z'].to_dict() #Los bloques se orientan de abajo hacia arriba, el bloque 0 es el que esta más abajo, 784 bloques
      self.L_b = self.database['Ton'].to_dict() #openPitBlockTonnage
      self.o_b = self.database['Mineral'].to_dict() #openPitBlockMineral
      self.openPitBlockRecovery = self.database['Recuperación'].to_dict() #openPitBlockRecovery
      self.openPitCopperLaw = self.database['%Cu'].to_dict() #openPitCopperLaw
      self.c_pbt = self.database['CPlanta CA'].to_dict() #openPitPlantCapacity
      self.c_mbt = self.database['CMina CA'].to_dict() #openPitMineCapacity

   def getOpenPitInfo(self):
      self.openPitBlocks = [i for i in range(len(self.openPitBlocksLength.values()))]

   def setOpenPitParameters(self):
      #OpenPit Parameters
      self.t_C   = {period : period + 1 for period in range(self.numberOfPeriods)}
      self.RMu_t = {period : 2 * 25806600.0/1 for period in range(self.numberOfPeriods)}#Superior infinita, 0 por abajo Originales: 13219200
      self.RMl_t = {period : 0.0/3 for period in range(self.numberOfPeriods)}#Valor original 8812800.0
      self.RPu_t = {period : 2 * 17777880.0/1 for period in range(self.numberOfPeriods)}#Valor original 10933380.0
      self.RPl_t = {period : 0/3 for period in range(self.numberOfPeriods)}#Valor original 7288920.0 
      self.qu_t  = {period : 1 for period in range(self.numberOfPeriods)}#Leyes promedio maxima y minima.
      self.ql_t  = {period : 0.0001 for period in range(self.numberOfPeriods)}
      self.delta = {period: 0 for period in range(self.numberOfPeriods)}
      self.maxTimeOpenPit = self.t_C[max(self.t_C)]

   def setOpenPitMineLimits(self):
      self.openPitBlocksLengthLimits = getNumberOfBlocksInADimension(self.openPitBlocksLength)
      self.openPitBlocksWidthLimits = getNumberOfBlocksInADimension(self.openPitBlocksWidth)
      self.openPitBlocksHeightLimits = getNumberOfBlocksInADimension(self.openPitBlocksHeight)
      self.predecessorBlock = self.setPredecessorBlocks()

   def setPredecessorBlocks(self):
      self.predecessorBlocks = finalBlock(self.openPitBlocks, self.openPitBlocksLengthLimits,self.openPitBlocksWidthLimits, self.openPitBlocksHeightLimits)
  
   def setPossibleHeights(self):
      self.blockHeight, self.maxHeight, self.minHeight, self.numOfDifferentsBlocks = self.openPitBlocksHeightLimits
   
   def setHeightSets(self):
      #Acá hay que redifinir self.B_v para que tenga las alturas de maxheight al sumarle el safetylvl

      self.V = [height for height in chain(range(self.minHeight,self.maxHeight,self.blockHeight), [self.maxHeight])]
      self.B_v = {}
      self.rho_v = {v:( ((v- self.safetyLevel - self.minHeightUnderground)/(self.maxHeightUnderground - self.minHeightUnderground)) if v - self.minHeightUnderground > 0 else 0 ) for v in self.V}

      for v in self.V:
         numberOfBlocksBelowV = (self.openPitBlocksLengthLimits[3]*self.openPitBlocksWidthLimits[3])*((v-self.minHeight)/self.openPitBlocksHeightLimits[0])
         blocksBelowV = [block for block in range(int(numberOfBlocksBelowV)) if not numberOfBlocksBelowV == 0]
         self.B_v[v] = blocksBelowV

   def setModel(self, isFinalIteration = False):#,w_opt):
      self.openPitModel = gp.Model(name = 'Modelo openPit')
      self.openPitModel.Params.TimeLimit = 3600
      self.openPitModel.Params.OutputFlag = 0
      
      # Open Pit tengo la variable del modelo cielo abierto continua
      self.x_bt = self.openPitModel.addVars(self.t_C, self.openPitBlocks, vtype=GRB.CONTINUOUS, name="x_b")
      
      #1. Restricci ́on sobre la cantidad de tonelaje m ́axima y m ́ınima a extraer en cada periodo.
      openPitModel_Ton_Up  = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.L_b[b] for b in self.openPitBlocks) 
                              <= self.RMu_t[ti] for ti in self.t_C), "Ton_max")
      openPitModel_Ton_low = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.L_b[b] for b in self.openPitBlocks) 
                              >= self.RMl_t[ti] for ti in self.t_C), "Ton_min")

      #2. Restricci ́on sobre la cantidad de material m ́axima y m ́ınima a extraer en cada periodo.
      openPitModel_Mat_Up_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.o_b[b] for b in self.openPitBlocks) <= 
                              self.RPu_t[ti] for ti in self.t_C), "Mat_max")
      openPitModel_Mat_low_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.o_b[b] for b in self.openPitBlocks) >= 
                              self.RPl_t[ti] for ti in self.t_C), "Mat_min")

      #3. Restricci ́on de precedencia de los bloques a extraer, debemos extraer los 5 bloques superiores al bloque objetivo para sacar a este
      
      #BLOCK_SUP_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[s, self.predecessorBlock[l][0]]*(self.maxTimeOpenPit-s+1) for s in self.t_C) <= 
      #                              gp.quicksum(self.x_bt[s, self.predecessorBlock[l][1]]*(self.maxTimeOpenPit-s+1) for s in self.t_C)  
      #                           for l in range(len(self.predecessorBlock))), "Superior_Block")
      openPitModel_Precedence = self.openPitModel.addConstrs(gp.quicksum(self.x_bt[s,a] for s in range(0,ti+1)) >= self.x_bt[ti, b] for b in self.openPitBlocks for ti in self.t_C for a in self.predecessorBlocks[b])


      #4. Restricci ́on sobre la ley m ́axima y m ́ınima por periodo.
      openPitModel_GQC_Up_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.L_b[b]*self.openPitCopperLaw[b] for b in self.openPitBlocks) <=
                           self.qu_t[ti] * gp.quicksum(self.x_bt[ti, b]*self.L_b[b] for b in self.openPitBlocks) for ti in self.t_C), 
                              "openPitModel_GQC_Up")

      openPitModel_GQC_low_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b]*self.L_b[b]*self.openPitCopperLaw[b] for b in self.openPitBlocks) >=
                           self.ql_t[ti] * gp.quicksum(self.x_bt[ti, b]*self.L_b[b] for b in self.openPitBlocks) for ti in self.t_C), 
                              "openPitModel_GQC_LOW")

      #5. Podemos extraer el bloque en un solo periodo.
      openPitModel_Reserve_cons_OP = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for ti in self.t_C) <= 1 for b in self.openPitBlocks), 
                              "openPitModel_Reserve_cons")

      #Función objetivo
      self.openPitObjectiveFunction = gp.quicksum(self.x_bt[ti, b]*((((self.basePrice*self.openPitCopperLaw[b]-self.c_pbt[b])*self.o_b[b])-(self.c_mbt[b]*self.L_b[b]))/((1+self.desc)**self.t_C[ti]))
                  for ti in self.t_C for b in self.openPitBlocks)
   
      self.openPitModel.setObjective(self.openPitObjectiveFunction, GRB.MAXIMIZE)
      self.openPitModel.Params.MIPGap = 0.01
      
      
      runtime = self.openPitModel.Runtime


   def execute(self, estimatedW_v, estimatedU_t):
      #Ojo que esta restricción quizás no es equivalente a sumar sobre todo0s los bloques y multiuplicar el lado derecho por la cantidad de bloques debajo de B_v
      #     self.heightRestriction = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for b in self.B_v[v]) <= (1 - estimatedW_v[v]) for v in self.V for ti in self.t_C), "heightRestriction")
      self.setModel()
      #print(f'W_v estimado: {estimatedW_v}\nU_t estimado: {estimatedU_t}')
      #self.heightRestriction = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for b in self.B_v[v]) <= (len(self.B_v[v]) * (1 - estimatedW_v[v])) for v in self.V for ti in self.t_C), "heightRestriction")
      self.heightRestriction = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for ti in self.t_C) <= (1 - estimatedW_v[v]) for v in self.V for b in self.B_v[v]), "heightRestriction")

      self.periodRestriction = self.openPitModel.addConstrs((gp.quicksum(self.x_bt[ti, b] for b in self.openPitBlocks ) <= (len(self.openPitBlocks) * (1 - estimatedU_t[ti])) for ti in self.t_C), "timeRestriction")

      self.openPitModel.optimize()


      #objVal = self.openPitModel.objVal

      if self.openPitModel.Status == gp.GRB.OPTIMAL:
         print("La solución del subproblema es óptima")
         objVal = self.openPitObjectiveFunction.getValue()

      else:
         objVal = 0

         print("El estado de la solución no es óptimo")
      #print(self.pi_bDict)
      #print(self.sigma_bDict)
      return objVal

In [7]:
class Main:
    def __init__(self, undergroundMineDataframe, openPitDataframe):
        self.openPitDataframe = openPitDataframe
        self.undergroundMineDataframe = undergroundMineDataframe
        self.numberOfPeriods = 10
        self.epsilon = 1
        self.safetyLevel = 60
        self.colHeight = 790#300 max 630 altura total de los drawpoints
        self.minColHeight = 0.40
        self.pos_x = 440#440#Coordenada x desde donde empezamos a extraer     
        self.pos_y = 550#550#Coordenada y desde donde empezamos a extraer
        self.pos_z = 530#780#Coordenada z desde donde empezamos a extraer     
        self.pos_x_f = 720#720#Coordenada x hazta donde extrameos  
        self.pos_y_f = 910#910#Coordenada y hazta donde extrameos
        
    def execute(self):
        self.createModels()
        self.setMasterProblemCrownPillarHeights()
        self.getResults()
    
    def createModels(self):
        self.createSubProblem()
        self.createMasterProblem()

    def createSubProblem(self):
        self.SubProblem = SubProblem(self.openPitDataframe, self.pos_z, self.pos_z + self.colHeight ,self.numberOfPeriods, self.safetyLevel)
        self.SubProblem.setParameters()

    def createMasterProblem(self):
        self.MasterProblem = MasterProblem(self.undergroundMineDataframe, self.numberOfPeriods,self.colHeight,self.minColHeight ,self.pos_x, self.pos_y, self.pos_z, self.pos_x_f, self.pos_y_f)
        self.MasterProblem.setParameters()
        
    def setMasterProblemCrownPillarHeights(self):
        self.MasterProblem.V, self.MasterProblem.rho_v = self.SubProblem.V, self.SubProblem.rho_v
        self.MasterProblem.B_v = self.SubProblem.B_v
        
    def getResults(self):
        self.subProblemObjValues = {}
        self.masterProblemObjValues = {}
        self.masterProblemObjValues2 = {}
        self.objValues = {}
        for v in self.MasterProblem.V:
            fixed_w_v = dict.fromkeys(self.MasterProblem.V,0)
            fixed_w_v[v] = 1
            for t in range(0, len(self.MasterProblem.t_S) + 1):
                fixed_u_t = dict.fromkeys(self.MasterProblem.t_S,1)
                for time in fixed_u_t:
                    if time < t:
                        fixed_u_t[time] = 0                
                subProblemObjValue = self.SubProblem.execute(fixed_w_v, fixed_u_t)
                masterProblemObjValue = self.MasterProblem.optimizeModel(fixed_w_v, fixed_u_t, subProblemObjValue)
                #masterProblemObjValue2 = self.MasterProblem.optimizeModel(fixed_w_v, fixed_u_t, 0)
                objValue = masterProblemObjValue + subProblemObjValue
                self.masterProblemObjValues[v,t] = masterProblemObjValue
                #self.masterProblemObjValues2[v,t] = masterProblemObjValue2 + subProblemObjValue
                self.subProblemObjValues[v,t] = subProblemObjValue
                self.objValues[v,t] = masterProblemObjValue + subProblemObjValue
                #print(v,t,masterProblemObjValue ,masterProblemObjValue2 + subProblemObjValue)
                print(v,t,objValue)
                print()

        print(self.masterProblemObjValues)
        print()
        print()
        print(self.subProblemObjValues)
        print()
        print()
        print(self.objValues)
    

In [8]:
main2 = Main(undergroundMineDataframe, openPitDataframe)
main2.execute()

3391
Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-02
Set parameter TimeLimit to value 3600
La solución del subproblema es óptima
Set parameter TimeLimit to value 3600
The model cannot be solved because it is unbounded
530 0 0.0

Set parameter TimeLimit to value 3600
La solución del subproblema es óptima
Set parameter TimeLimit to value 3600
The model cannot be solved because it is unbounded
530 1 553956292.8973836

Set parameter TimeLimit to value 3600
La solución del subproblema es óptima
Set parameter TimeLimit to value 3600
The model cannot be solved because it is unbounded
530 2 1553997731.7239718

Set parameter TimeLimit to value 3600
La solución del subproblema es óptima
Set parameter TimeLimit to value 3600
The model cannot be solved because it is unbounded
530 3 2460023540.917484

Set parameter TimeLimit to value 3600
La solución del subproblema es óptima
Set parameter TimeLimit to value 3600
The model cannot be solved because it is un

5 periodos 25 min  48 secs (1427121198.3897185, 0.0)

(930, 0) 1086862208.548721
(970, 0) 1206745460.300233
(1010, 0) 1332149501.0003462
(1050, 0) 1375328262.741136
(1090, 0) 1388540092.2633448
(1130, 0) 1399779509.5479581
(1170, 0) 1407584538.3235626
(1210, 0) 1413208669.9559355
(1250, 0) 1422844994.968577
(1290, 0) 1427121198.3897185

10 periodos 151 min 52.9 segs de iteración 1% gap (3051987965.3455014, 2309039490.159496)


Master problem obj1
(930, 0) 1097311105.2106926
(930, 1) 1550282561.628895
(930, 2) 2418513958.613047
(930, 3) 2993416495.951757
(930, 4) 3051987965.3455014


15 periodos 433 min 46.5 segs de iteracion 
Master problem obj1

(930, 0) 1103824556.1167746
(930, 1) 1552576348.8013976
(930, 2) 2436181741.2659087
(930, 3) 2991180334.7058
(930, 4) 3021619556.360697


530 15 0 3525862445.174612
530 15 3525862445.174612

1290 15 0 99250319.06612594
1290 15 99250319.06612594


415 min se demora en iterar sobre todos los periodos de una altura.

In [13]:
maxValue = 0   
comb = ""
for i in main2.masterProblemObjValues:
    if main2.masterProblemObjValues[i] > maxValue:
        maxValue = main2.masterProblemObjValues[i]
        comb = i
        print(comb,maxValue)

(930, 0) 1097311105.2106926
(930, 1) 1550282561.628895
(930, 2) 2418513958.613047
(930, 3) 2993416495.951757
(930, 4) 3051987965.3455014


In [10]:
maxValue = 0   
comb = ""
for i in main2.masterProblemObjValues2:
    if main2.masterProblemObjValues2[i] > maxValue:
        maxValue = main2.masterProblemObjValues2[i]
        comb = i
        print(comb,maxValue)

5 periodos 24 min 4.2 segs
(930, 0) 1086862208.548721
(970, 0) 1206745460.300233
(1010, 0) 1332149501.0003462
(1050, 0) 1375328262.741136
(1090, 0) 1388540092.2633448
(1130, 0) 1399779509.5479581
(1170, 0) 1407584538.3235626
(1210, 0) 1413208669.9559355
(1250, 0) 1422844994.968577
(1290, 0) 1427121198.3897185

In [14]:
main2.masterProblemObjValues[(930, 4)], main2.subProblemObjValues[(930, 4)]

(3051987965.3455014, 2309039490.159496)

In [12]:
main2.masterProblemObjValues[(985, 1)], main2.subProblemObjValues[(985, 1)]

KeyError: (985, 1)

In [ ]:
main2.objValues

{(745, 0): 0.0,
 (745, 1): 1331535302.3718894,
 (745, 2): 2596883876.744168,
 (745, 3): 3032053282.561581,
 (745, 4): 3177392487.6525445,
 (745, 5): 3230527593.1668177,
 (745, 6): 3252633442.1429214,
 (745, 7): 3261902147.1637473,
 (745, 8): 3265761849.522701,
 (745, 9): 3267632927.779926,
 (745, 10): 3268577931.0196896,
 (785, 0): 0.0,
 (785, 1): 1331535302.3718894,
 (785, 2): 2557208887.0510983,
 (785, 3): 2959366381.0415363,
 (785, 4): 3087430989.376214,
 (785, 5): 3134076097.4902987,
 (785, 6): 3152696733.665735,
 (785, 7): 3160542036.3106403,
 (785, 8): 3164233541.3156714,
 (785, 9): 3165997258.5447345,
 (785, 10): 3166816334.4144926,
 (825, 0): 0.0,
 (825, 1): 1331535302.3718894,
 (825, 2): 2507510955.1178093,
 (825, 3): 2873199751.5605803,
 (825, 4): 2991243871.147936,
 (825, 5): 3030999864.2055936,
 (825, 6): 3046915079.476129,
 (825, 7): 3053397480.3653984,
 (825, 8): 3056427101.23759,
 (825, 9): 3057956400.963081,
 (825, 10): 3058635403.52531,
 (865, 0): 0.0,
 (865, 1): 13315

In [ ]:
main2.MasterProblem.B_v

{745: [],
 785: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  1

In [ ]:
a = {}
for v in main2.SubProblem.V:
    numberOfBlocksBelowV = (main2.SubProblem.openPitBlocksLengthLimits[3]*main2.SubProblem.openPitBlocksWidthLimits[3])*((v-main2.SubProblem.minHeight)/main2.SubProblem.openPitBlocksHeightLimits[0])
    a[v] = numberOfBlocksBelowV

    blocksBelowV = [block for block in range(int(numberOfBlocksBelowV)) if not numberOfBlocksBelowV == 0]


In [ ]:
a

{745: 0.0,
 785: 784.0,
 825: 1568.0,
 865: 2352.0,
 905: 3136.0,
 945: 3920.0,
 985: 4704.0,
 1025: 5488.0,
 1065: 6272.0,
 1105: 7056.0,
 1145: 7840.0,
 1185: 8624.0,
 1225: 9408.0,
 1265: 10192.0,
 1305: 10976.0}